In [1]:
import numpy as np
from qibo import Circuit, gates

In [7]:
# generate an empty quantum circuit with 5 qubits
nqubits = 5
circ = Circuit(nqubits)
print(circ.draw())

q0: ─
q1: ─
q2: ─
q3: ─
q4: ─


In [8]:
def initialize_uniform_dist_Hadammads(nqubits,ancilla=0):
    """
    This function adds Hadammard gates to all the qubits in a circuit.

    Inputs:
    ------
        - nqubits: `int`. Number of qubits of the circuit
        - ancilla: `int`. Number of ancilla qubits. No H gate is applied to them.
    Outputs:
    ------
        -circuit_init: `circuit` with a Hadamard gate applied to each qubit
        
    """

    circuit_init = Circuit(nqubits+ancilla)  # start an empty circuit of nqubits
    circuit_init.add([gates.H(i) for i in range(nqubits)])  # add a H to each qubit, inline loop

    return circuit_init

In [9]:
circ = circ + initialize_uniform_dist_Hadammads(nqubits) #adding circuits
print(circ.draw())

q0: ─H─
q1: ─H─
q2: ─H─
q3: ─H─
q4: ─H─


In [10]:
list_desired = [30, 15, 1]  # list of ints to amplify

In [11]:
def list_to_binary(nqubits, list_desired):
    """
    This functions translates a list of ints to binary.
    These ints are the numbers that will get amplified by the Grover's algorithm when using the trivial oracle.

    Inputs:
    ------
        - nqubits: `int`. Number of qubits of the circuit
        - list_desired: `list` of `ints`. The binary form of each int must fit in nqubits bits
        
    Outputs:
    ------
        - binary_list: `list` of `str` with the binary form of the ints in list_desired
        
    """

    binary_list = [] #empty lsit
    for b in list_desired:
        if len(bin(b))-2 > nqubits: #looks if each number in list_desired can fit in nqubits
            raise ValueError("list_desired does not fit in the number of qubits")
        else: #generates a list of strings with the binary form of each int in list_desired
            binary_list.append(bin(b)[2:].zfill(nqubits))

    return binary_list

In [12]:
list_binary = list_to_binary(nqubits,list_desired)
print(list_binary)

['11110', '01111', '00001']


In [13]:
def oracle_trivial(nqubits, list_desired):
    """
    This functions generates a Grover oracle that phase-tags a list of binary numbers.
    Each number get tagged by a CZ gate applied to all qubits and X gates applied before and after the CZ to the qubits corresponding with '0's in each binary number.

    Inputs:
    ------
        - nqubits: `int`. Number of qubits of the circuit
        - list_desired: `list` of `ints`. The binary form of each int must fit in nqubits bits

    Outputs:
    ------
        - oracle: `circuit`, that phase-tags the desired states
        
    """
    binary_list=list_to_binary(nqubits,list_desired)
    oracle = Circuit(nqubits)  #starts an empty circuit of nqubits

    for b in binary_list:
        for j, i in enumerate(b): 
            #add a X gate to each qubit corresponding with '0' in the binary number
            if i == "0": 
                oracle.add(gates.X(j))

        oracle.add(gates.Z(nqubits - 1).controlled_by(*range(nqubits)[0 : len(range(nqubits - 1))]))  
        # CZ to all qubits. Target qubit election does not matter

        for j, i in enumerate(b): 
            #add a X gate to each qubit corresponding with '0' in the binary number
            if i == "0":
                oracle.add(gates.X(j))

    return oracle

In [14]:
oracle = oracle_trivial(nqubits, list_desired)
print(oracle.draw())

q0: ───o─X─o─X─X─o─X─
q1: ───o───o─X───o─X─
q2: ───o───o─X───o─X─
q3: ───o───o─X───o─X─
q4: ─X─Z─X─Z─────Z───


In [15]:
def grover_diffuser(nqubits,ancilla=0):
    """
    This functions generates the Grover diffuser circuit for nqubits. 

    Inputs:
    ------
        - nqubits: `int`. Number of qubits of the circuit
        - ancilla: `int`. Optional. Number of ancilla qubits, no gate is applied to them. 
    Outputs:
    ------
        - diffuser: `Circuit`. A qibo implementation of the Grover diffuser
        
    """
    diffuser = Circuit(nqubits+ancilla)  # empty circuit of nqubits+ancilla


    #this function assumes the ancilla qubits will be the last in the circuit, should me changed otherwise
    diffuser.add([gates.H(i) for i in range(nqubits)])
    diffuser.add([gates.X(i) for i in range(nqubits)])
    # added H and X gates to all qubits

    diffuser.add(gates.Z(nqubits - 1).controlled_by(*range(nqubits)[0 : len(range(nqubits - 1))]))
    # CZ to all qubits. Target qubit election does not matter

    diffuser.add([gates.X(i) for i in range(nqubits)])
    diffuser.add([gates.H(i) for i in range(nqubits)])
    # added X and H gates to all qubits

    return diffuser

In [16]:
print(grover_diffuser(nqubits).draw())
type(grover_diffuser(5))

q0: ─H─X─o─X─H─
q1: ─H─X─o─X─H─
q2: ─H─X─o─X─H─
q3: ─H─X─o─X─H─
q4: ─H─X─Z─X─H─


qibo.models.circuit.Circuit

In [17]:
def Iterations_T(N, M):
    """
    This functions calculates the number of iterations of Oracle+Diffuser required to amplify the desired states

    Inputs
    ------
        - N: `int`. It is the total number of states, usually, N=2**nqubits
        - M: `int`. It is the number of states to amplify. In this case, M=len(list_desired)
        
    Outputs:
    ------
        - num_int: `int`. The required number of repetitions of the pair Oracle+Diffuser needed to reach maximun amplification
    """

    num_it=int(np.pi / 4 * np.sqrt(N / M))  # rounding by truncation

    return num_it

In [18]:
num_it = Iterations_T(N=2**nqubits, M=len(list_desired))
print(num_it)

2


In [20]:
def grover_iterator(num_it, oracle, diffuser):
    """'
    This functions returns a circuit whith the correct number of iteratios of the pair oracle+diffuser.
    The number of qubits of the oracle and the diffuser must be the same.

    Inputs:
    ------
        - num_it: `int`. Output of Iterations_T(N,M)
        - oracle: `circuit`. Grover oracle that phase-tags desired states
        - diffuser: `circuit`. Grover diffuser

    Outputs:
    ------
        - grover: `circuit` with the required repetitions of oracle+diffuser

    """
    if oracle.nqubits == diffuser.nqubits: #check if the number of qubits of the oracle and the diffuser is the same
        grover = Circuit(oracle.nqubits) #creates an empty circuit of nqubits
    else: #raises an error if the number of qubits of the oracle and the diffuser is not the same
        raise ValueError("Error: not the same number of qubits in oracle and diffuser")
    
    for i in range(num_it): #repeats the oracle+diffuser the required amount of times
        grover = grover + oracle + diffuser
    return grover

In [21]:
### a complete example of this Grover's algorithm implementation

nqubits = 6 #number of qubits
list_desired = [*range(0,int(2**nqubits/4),1)] #states to amplify

circ_complete = initialize_uniform_dist_Hadammads(nqubits) + grover_iterator(
    num_it=Iterations_T(N=2**nqubits, M=len(list_desired)),
    oracle=oracle_trivial(nqubits, list_desired),
    diffuser=grover_diffuser(nqubits),
) #circuit with inizialization and the correct number of repetitions of oracle+diffuser

# add measures
circ_complete.add([gates.M(i) for i in range(nqubits)])

#print(circ_complete.draw())

In [24]:
# run the simulation
result = circ_complete(nshots=2000000)
print(result.frequencies(binary=False))

Counter({2: 127611, 11: 127248, 5: 126545, 8: 126304, 1: 125498, 7: 125354, 12: 125163, 0: 125007, 4: 124541, 9: 124483, 14: 124114, 6: 124112, 10: 124106, 3: 123499, 13: 123482, 15: 122933})


In [25]:
#### now, with the qibo functions for the grover's algorithm

In [26]:
def matrix_unitary(nqubits, list_desired):
    """
    This function generates an unitary matrix which phase-tags the desired quantum states.
    This matrix is the mathematical representation of a Grover oracle.
    The output matrix is square with dimensions of (2**nqubits, 2**nqubits)

    Inputs:
    ------
        - nqubits: `int`. Number of qubits.
        - list_desired: `list` of `ints`. The max value must be <2**nqubits

    Outputs:
    ------
        - matrix: `numpy.ndarray`. The unitary matrix generated

    """

    N = 2**nqubits 

    v = [1] * N

    if max(list_desired) >= N:

        raise ValueError(
            "Error: some desired value does not fit in the number of qubits selected"
        )

    for i in range(N):

        if i in list_desired:

            v[i] = -1  # pi-phase marking

    matrix = np.diag(v)

    return matrix

In [27]:
def oracle_unitary(nqubits, list_desired):
    """
    This function generates a Grover oracle that phase-tags the desired values.
    It uses the `gates.Unitary()` method. To do so, an unitary matrix is generated by `matrix_unitary` `function`.
    An ancilla qubit is added as the qibo function Grover() needs the oracle to have 1 ancilla qubit. No gate is applied to this qubit.

    Inputs:
    ------
        - nqubits: `int`. Number of qubits.
        - list_desired: `list` of `ints`. The max value must be <2**nqubits

    Outputs:
    ------
        - orace: `circuit`, that phase-tags the desired states

    """

    matrix = matrix_unitary(nqubits, list_desired)
    oracle = Circuit(nqubits + 1) #this +1 is needed to use Grover() later

    oracle.add([gates.Unitary(matrix, *range(nqubits))])
    return oracle

In [28]:
from qibo.models import Grover

In [29]:
#example of the Grover's algorithm completely automated by using the qibo functions
nqubits=7
list_desired=[2,9,6] 
oracle_1=oracle_unitary(nqubits,list_desired)
grover = Grover(oracle_1, superposition_circuit=initialize_uniform_dist_Hadammads(nqubits), number_solutions=len(list_desired))
result_grover=grover.execute(nshots=20000,freq=True)
print(result_grover)


(['0000010', '0001001', '0000110'], 5)


In [33]:
# Grover's algorithm to solve the 2x2 sudoku:

In [34]:
def Xor_2qubits(circuit,q1,q2,ancilla):
    """
    This function add gates to a circuit equivalent to a XOR. 
    Two CX gates are added to the circuit controlled by q1 and q2, both targeting the ancilla qubit

    Inputs:
    ------
        - circuit: `circuit` to add the gates
        - q1: `int`, number of the first control qubit
        - q2: `int`, number of the second control qubit
        - ancilla: `int`, number of the target qubit

    Outputs:
    ------
        - circuit: `circuit`with the XOR equivalent gates added

    """
    circuit.add(gates.X(ancilla).controlled_by(q1))
    circuit.add(gates.X(ancilla).controlled_by(q2))
    return circuit

In [35]:
test=Circuit(3)
circuit=Xor_2qubits(test,0,1,2)
print(circuit.draw())

q0: ─o───
q1: ─|─o─
q2: ─X─X─


In [36]:
def sudoku_oracle(nqubits,clause_list):
    """
    This function creates a Grover oracle that phase-tags the solutions to the sudoku.

    Inputs:
    ------
        - nqubits: `int`
        - clause_list: `list` of `tuple`

    Outputs:
    ------
        - oracle: `circuit` that phase-tags the desired states

    """

    nancilla = len(clause_list)
    total_qubits = nqubits+nancilla
    if nqubits != nancilla:
            raise ValueError("Error: the number of qubits must be equal than the number of clauses")
    
    
    oracle = Circuit(nqubits+nancilla)

    for j,clause in enumerate(clause_list):
        Xor_2qubits(oracle,clause[0], clause[1], nqubits+j)
    
    oracle.add(gates.Z(total_qubits - 1).controlled_by(*range(total_qubits)[nqubits : len(range(total_qubits - 1))]))
        
    for j,clause in enumerate(clause_list):
        Xor_2qubits(oracle,clause[0], clause[1], nqubits+j)

    return oracle

In [37]:
clause_list = [ (0,1), (0,2), (1,3), (2,3)]
print(sudoku_oracle(4,clause_list).draw())

q0: ─o───o─────────────o───o───────────
q1: ─|─o─|───o─────────|─o─|───o───────
q2: ─|─|─|─o─|───o─────|─|─|─o─|───o───
q3: ─|─|─|─|─|─o─|─o───|─|─|─|─|─o─|─o─
q4: ─X─X─|─|─|─|─|─|─o─X─X─|─|─|─|─|─|─
q5: ─────X─X─|─|─|─|─o─────X─X─|─|─|─|─
q6: ─────────X─X─|─|─o─────────X─X─|─|─
q7: ─────────────X─X─Z─────────────X─X─


In [38]:
#example of grover's algorithm for the sudoku problem
nqubits = 4 #number of qubits


circ_complete = initialize_uniform_dist_Hadammads(nqubits,ancilla=len(clause_list)) + grover_iterator(
    num_it=Iterations_T(N=2**nqubits, M=1),
    oracle=sudoku_oracle(nqubits,clause_list),
    diffuser=grover_diffuser(nqubits,ancilla=len(clause_list)),
) #circuit with inizialization and the correct number of repetitions of oracle+diffuser

# add measures
circ_complete.add([gates.M(i) for i in range(nqubits)])
print(circ_complete.draw())

q0:     ─H─o───o─────────────o───o───────────H─X─o─X─H─o───o─────────────o───o ...
q1:     ─H─|─o─|───o─────────|─o─|───o───────H─X─o─X─H─|─o─|───o─────────|─o─| ...
q2:     ─H─|─|─|─o─|───o─────|─|─|─o─|───o───H─X─o─X─H─|─|─|─o─|───o─────|─|─| ...
q3:     ─H─|─|─|─|─|─o─|─o───|─|─|─|─|─o─|─o─H─X─Z─X─H─|─|─|─|─|─o─|─o───|─|─| ...
q4:     ───X─X─|─|─|─|─|─|─o─X─X─|─|─|─|─|─|───────────X─X─|─|─|─|─|─|─o─X─X─| ...
q5:     ───────X─X─|─|─|─|─o─────X─X─|─|─|─|───────────────X─X─|─|─|─|─o─────X ...
q6:     ───────────X─X─|─|─o─────────X─X─|─|───────────────────X─X─|─|─o────── ...
q7:     ───────────────X─X─Z─────────────X─X───────────────────────X─X─Z────── ...

q0: ... ───────────H─X─o─X─H─o───o─────────────o───o───────────H─X─o─X─H─M─
q1: ... ───o───────H─X─o─X─H─|─o─|───o─────────|─o─|───o───────H─X─o─X─H─M─
q2: ... ─o─|───o───H─X─o─X─H─|─|─|─o─|───o─────|─|─|─o─|───o───H─X─o─X─H─M─
q3: ... ─|─|─o─|─o─H─X─Z─X─H─|─|─|─|─|─o─|─o───|─|─|─|─|─o─|─o─H─X─Z─X─H─M─
q4: ... ─|─|─|─|─|───────────X─

In [39]:
result = circ_complete(nshots=2000000)
print(result.frequencies(binary=False))

Counter({6: 330211, 9: 329597, 12: 96248, 2: 96125, 13: 96104, 1: 95978, 11: 95885, 8: 95742, 7: 95703, 5: 95659, 14: 95632, 4: 95574, 0: 95550, 3: 95531, 10: 95510, 15: 94951})
